In [72]:
import requests
from tqdm.auto import tqdm
import pandas as pd
import ssl
import nltk
import time
import math
import glob
import pymorphy2
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Раздел 1: Разработка и анализ

## Сбор вакансий

In [2]:
# определим работодателей для сбора вакансий
employers = {'name': ['sber', 'sbertech', 'rosatom', 'gazprom_neft'], 'id': [3529, 906557, 577743, 39305]}

num_per_page = 100
all_vacancy_ids = []

# соберем идентификаторы вакансий по каждому работодателю
for employer in employers['id']:
    url = f'https://api.hh.ru/vacancies?employer_id={employer}&per_page={num_per_page}'
    res = requests.get(url)
    number_of_pages = res.json().get('pages')
    
    for i in tqdm(range(number_of_pages)):
        url = f'https://api.hh.ru/vacancies?employer_id={employer}&page={i}&per_page={num_per_page}'
        res = requests.get(url)
        vacancies = res.json()
        vacancy_ids = [el.get('id') for el in vacancies.get('items')]
        all_vacancy_ids.extend(vacancy_ids)
        

100%|███████████████████████████████████████████| 18/18 [00:08<00:00,  2.14it/s]


In [3]:
# число вакансий
print(len(all_vacancy_ids))

3919


In [4]:
# проверим состав получаемых полей
url = f'https://api.hh.ru/vacancies/54162408'

r = requests.get(url)

# нормализуем для удобства
j = pd.json_normalize(r.json(), max_level=1)

print(j.columns)

Index(['id', 'premium', 'relations', 'name', 'insider_interview',
       'response_letter_required', 'address', 'allow_messages', 'contacts',
       'description', 'branded_description', 'vacancy_constructor_template',
       'key_skills', 'accept_handicapped', 'accept_kids', 'archived',
       'response_url', 'specializations', 'professional_roles', 'code',
       'hidden', 'quick_responses_allowed', 'driver_license_types',
       'accept_incomplete_resumes', 'published_at', 'created_at',
       'negotiations_url', 'suitable_resumes_url', 'apply_alternate_url',
       'has_test', 'test', 'alternate_url', 'working_days',
       'working_time_intervals', 'working_time_modes', 'accept_temporary',
       'billing_type.id', 'billing_type.name', 'area.id', 'area.name',
       'area.url', 'salary.from', 'salary.to', 'salary.currency',
       'salary.gross', 'type.id', 'type.name', 'site.id', 'site.name',
       'experience.id', 'experience.name', 'schedule.id', 'schedule.name',
       'emplo

In [277]:
# соберем данные по вакансиям

# рассчитаем число шагов для сбора (для минимизации шансов отключения сессии)
step = 200
number_of_steps = math.ceil(len(all_vacancy_ids)/step)

for stp in range(number_of_steps):
    all_vacancy_descriptions=pd.DataFrame(columns=['id', 'name', 'description', 'employer.name'])
    loop_ids = all_vacancy_ids[stp*step:(stp+1)*step]
    
    for i in tqdm(loop_ids):
        
        url = f'https://api.hh.ru/vacancies/{i}'
        res = requests.get(url)
        # нормализация json
        vacancies = pd.json_normalize(res.json(), max_level=1).filter(['id', 'name', 'description', 'employer.name'])
        
        all_vacancy_descriptions = pd.concat([all_vacancy_descriptions, vacancies], join="outer")
    
    # сохранение в csv
    all_vacancy_descriptions.to_csv(f'descriptions_{stp}.csv')
    # ожидание, чтобы минимизировать шансы на завершение сессии со стороны API hh.ru
    time.sleep(10)
    

  4%|█▌                                         | 7/200 [00:02<01:12,  2.65it/s]


KeyboardInterrupt: 

## Разработка алгоритма ранжирования

In [5]:
# загрузим сохраненные данные
df = pd.DataFrame()

for f in glob.glob('descriptions_*.csv'):
    df = pd.concat([df, pd.read_csv(f, index_col='id')], join="outer")
    
df = df.drop(columns='Unnamed: 0')

In [6]:
# проверим есть ли вакансии без описания
df[df.description.isna()]

,name,description,employer.name
id,,,


In [46]:
# скачаем стопслова
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

stopwords_rus = stopwords.words('russian')

# дополним лишними элементами (html, css)
stopwords_rus.extend(['<strong>', '</strong>', 'strong', '<ul>', '</ul>', 'ul', '<li>', '</li>', 'li', '<p>', '</p>', 'p', 'n', 'u'])


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ilyanaboyshchikov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
# загрузим описание опыта
with open('Описание опыта.txt', encoding='utf-8') as f:
    experience = pd.Series(f.read())

In [28]:
# объединим датафрейм с описанями вакансий и строку с опытом кандидата
description_with_experience = pd.concat([df.description, experience], join="outer")
description_with_experience = description_with_experience.reset_index(drop=True)

description_with_experience.iloc[[0, -1]].replace('\n', ' ').replace('\u2028',' ')

0       <p>Наша команда занимается развитием прикладного аналитического решения. Мы собираем данные, связанные с инвестиционной деятельностью дочерних компаний Банка, обрабатываем их и предоставляем пользователям в удобном и доступном виде.</p> <p>Мы ищем разработчика SQL, который может и хочет взять на себя решение задач развития витрин данных на PostgreSQL для формирования аналитической отчетности на Qlik Sense.</p> <p><strong>Мы ожидаем, что Вы:</strong></p> <ul> <li>Имеете большой опыт работы с SQL (преимущественно PostgreSQL): написание сложных запросов, создание процедур и функций, оптимизации кода, изменения чужого кода</li> <li>Имеете опыт работы с документацией (требования, логическая модель данных)</li> <li>Имеете опыт общения с бизнес-пользователями, описания требований</li> <li>Знакомы с принципами построения и типовой архитектурой хранилищ данных, имеете опыт проектирования хранилищ данных, витрин данных</li> <li>Имеете опыт работы в проектах внедрения и развития хранилищ 

In [51]:
# функция лемматизации
def lemmatize(row):
    morph = pymorphy2.MorphAnalyzer()
    t = []
    for word in row.split():
        if word in ['strong', 'ul', 'li']:
            t.append('')
        elif len(word)<=3:
            continue
        else:
            p = morph.parse(word)[0]
            t.append(p.normal_form)
    return " ".join(t)

# применим к датасету
description_with_experience = description_with_experience.apply(lemmatize)

In [52]:
# настроим TF-IDF
text_transformer = TfidfVectorizer(stop_words=stopwords_rus, ngram_range=(1,1), lowercase=True, max_features=10000)
text = text_transformer.fit_transform(description_with_experience)
text

<3920x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 521326 stored elements in Compressed Sparse Row format>

In [53]:
# посмотрим на полученный словарь обработанных слов
text_transformer.vocabulary_

{'наша': 4726,
 'команда': 3721,
 'заниматься': 3015,
 'развитие': 7077,
 'прикладной': 6558,
 'аналитический': 1519,
 'решения': 7471,
 'собирать': 8086,
 'данные': 2483,
 'связанный': 7743,
 'инвестиционный': 3236,
 'деятельность': 2597,
 'дочерний': 2800,
 'компания': 3770,
 'банка': 1657,
 'обрабатывать': 4981,
 'предоставлять': 6393,
 'пользователь': 6137,
 'удобный': 9100,
 'доступный': 2790,
 'виде': 1961,
 'искать': 3410,
 'разработчик': 7136,
 'sql': 1182,
 'который': 3969,
 'мочь': 4532,
 'хотеть': 9543,
 'взять': 1958,
 'решение': 7467,
 'задача': 2935,
 'витрина': 1992,
 'дать': 2493,
 'postgresql': 960,
 'формирование': 9438,
 'отчётность': 5475,
 'qlik': 998,
 'sense': 1123,
 'ожидаем': 5121,
 'иметь': 3215,
 'большой': 1767,
 'опыт': 5236,
 'работа': 7022,
 'преимущественно': 6462,
 'написание': 4639,
 'сложный': 8007,
 'запросов': 3053,
 'создание': 8155,
 'процедура': 6931,
 'функций': 9483,
 'оптимизация': 5228,
 'кода': 3687,
 'изменение': 3181,
 'документация': 2714

In [54]:
pd.set_option('display.max_colwidth', None)

# рассчитаем матрицу косинусных расстояний
cos_similarity_matrix = cosine_similarity(text)

# возьмем только столбец с расстояниями для опыта кандидата
cos_similarity_series = pd.Series(cos_similarity_matrix[-1])

# отбросим последнюю строку с расстоянием = 1 
cos_similarity_series = cos_similarity_series[:-1]

cos_similarity_series.index = df.index

# добавим полученные расстояния обратно в датасет
df['cos_similarity'] = cos_similarity_series[:-1]

# отсортируем по убыванию косинусного расстояния
df = df.sort_values(by='cos_similarity', ascending=False)

# отобразим 10 первых вакансий
df[['name', 'employer.name', 'cos_similarity']].head(10)

,name,employer.name,cos_similarity
id,,,
55846460,Разработчик SQL (PostgreSQL),Сбербанк,0.182849
55850116,BI Главный специалист,Газпром нефть,0.181157
66134564,Главный специалист по развитию и контролю сервиса,Газпром нефть,0.157170
53931712,ETL разработчик,Сбербанк,0.150187
55309191,ETL разработчик,Сбербанк,0.148819
55481572,Data Engineer | ETL разработчик (Spark),Сбербанк,0.148562
54923506,DevOps,Сбербанк,0.146579
55858523,Ведущий инженер по разработке QlikView \ Qlik Sense,Сбербанк,0.144511
66091343,Системный аналитик (Brokerage),Сбербанк,0.144243


# Раздел 2: Разработанные функции

In [63]:
# функция сбора вакансий
# принимает на вход список с идентификаторами компаний с сайта hh.ru
# одним из способов получить идентификатор - скопировать его из ссылки на любую вакансию интересующей компании 
def get_vacancies(employer_id_list):
    num_per_page = 100
    all_vacancy_ids = []

    # соберем идентификаторы вакансий по каждому работодателю
    for employer in employer_id_list:
        url = f'https://api.hh.ru/vacancies?employer_id={employer}&per_page={num_per_page}'
        res = requests.get(url)
        number_of_pages = res.json().get('pages')

        for i in tqdm(range(number_of_pages)):
            url = f'https://api.hh.ru/vacancies?employer_id={employer}&page={i}&per_page={num_per_page}'
            res = requests.get(url)
            vacancies = res.json()
            vacancy_ids = [el.get('id') for el in vacancies.get('items')]
            all_vacancy_ids.extend(vacancy_ids)
        
    # соберем данные по вакансиям

    # рассчитаем число шагов для сбора (для минимизации шансов отключения сессии)
    step = 200
    number_of_steps = math.ceil(len(all_vacancy_ids)/step)

    for stp in range(number_of_steps):
        all_vacancy_descriptions=pd.DataFrame(columns=['id', 'name', 'description', 'employer.name'])
        loop_ids = all_vacancy_ids[stp*step:(stp+1)*step]

        for i in tqdm(loop_ids):

            url = f'https://api.hh.ru/vacancies/{i}'
            res = requests.get(url)
            # нормализация json
            vacancies = pd.json_normalize(res.json(), max_level=1).filter(['id', 'name', 'description', 'employer.name'])

            all_vacancy_descriptions = pd.concat([all_vacancy_descriptions, vacancies], join="outer")

        # сохранение в csv
        all_vacancy_descriptions.to_csv(f'descriptions_{stp}.csv')
        # ожидание, чтобы минимизировать шансы на завершение сессии со стороны API hh.ru
        time.sleep(10)
    
    return None
        

# функция лемматизации (заимствованно с блока Data Science)
def lemmatize(row):
    morph = pymorphy2.MorphAnalyzer()
    t = []
    for word in row.split():
        if word in ['strong', 'ul', 'li']:
            t.append('')
        elif len(word)<=3:
            continue
        else:
            p = morph.parse(word)[0]
            t.append(p.normal_form)
    return " ".join(t)

# функция расчета и отображения ТОПа вакансий
# принимает на вход имя файла с описанием опыта кандидата 
# и число случайно выбираемых описаний вакансий
def get_top_vacancies(candidate_experience_file, number_of_random_rows):
    # загрузим сохраненные данные
    df = pd.DataFrame()

    for f in glob.glob('descriptions_*.csv'):
        df = pd.concat([df, pd.read_csv(f, index_col='id')], join="outer")

    df = df.drop(columns='Unnamed: 0')
    
    # выберем случайные N вакансий
    df = df.sample(number_of_random_rows, ignore_index=True)
    
    # скачаем стопслова
    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        pass
    else:
        ssl._create_default_https_context = _create_unverified_https_context

    nltk.download('stopwords')

    stopwords_rus = stopwords.words('russian')

    # дополним лишними элементами (html, css)
    stopwords_rus.extend(['<strong>', '</strong>', 'strong', '<ul>', '</ul>', 'ul', '<li>', 
                          '</li>', 'li', '<p>', '</p>', 'p', 'n', 'u'])
    
    # загрузим описание опыта
    with open(candidate_experience_file, encoding='utf-8') as f:
        experience = pd.Series(f.read())
        
    
    # объединим датафрейм с описанями вакансий и строку с опытом кандидата
    description_with_experience = pd.concat([df.description, experience], join="outer")
    description_with_experience = description_with_experience.reset_index(drop=True)

    description_with_experience.iloc[[0, -1]].replace('\n', ' ').replace('\u2028',' ')
    
    # применим функцию лемматизации к датасету
    description_with_experience = description_with_experience.apply(lemmatize)
    
    # настроим TF-IDF
    text_transformer = TfidfVectorizer(stop_words=stopwords_rus, ngram_range=(1,1), lowercase=True, max_features=10000)
    text = text_transformer.fit_transform(description_with_experience)
    
    pd.set_option('display.max_colwidth', None)

    # рассчитаем матрицу косинусных расстояний
    cos_similarity_matrix = cosine_similarity(text)

    # возьмем только столбец с расстояниями для опыта кандидата
    cos_similarity_series = pd.Series(cos_similarity_matrix[-1])

    # отбросим последнюю строку с расстоянием = 1 
    cos_similarity_series = cos_similarity_series[:-1]

    cos_similarity_series.index = df.index

    # добавим полученные расстояния обратно в датасет
    df['cos_similarity'] = cos_similarity_series[:-1]

    # отсортируем по убыванию косинусного расстояния
    df = df.sort_values(by='cos_similarity', ascending=False)

    # вернем в результате 10 наиболее подходящих вакансий
    return df[['name', 'employer.name', 'description', 'cos_similarity']].head(10)
  

## Вызов разработанных функций

In [64]:
# определим работодателей для сбора вакансий
employers = {'name': ['sber', 'sbertech', 'rosatom', 'gazprom_neft'], 'id': [3529, 906557, 577743, 39305]}

# используем функцию сбора данных
# get_vacancies(employers['id'])

# используем функцию Топа вакансий
result = get_top_vacancies('Описание опыта.txt', 100)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ilyanaboyshchikov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Анализ результатов

In [69]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words=stopwords_rus) 
X = vectorizer.fit_transform(result.description) 

features = pd.DataFrame(X.A, columns=vectorizer.get_feature_names_out())
type(features)

s = features.sum()
features_sort = pd.DataFrame(features[s.sort_values(ascending=False).index[:15]])

features_sort

,опыт,данных,работы,разработки,знание,greenplum,задач,это,команда,субд,процессов,дмс,разработке,участие,команде
0,14,7,11,8,1,1,2,2,1,1,2,1,3,1,1
1,3,5,5,0,3,0,0,0,1,0,2,1,0,1,0
2,6,4,7,4,1,1,2,2,1,1,1,1,2,1,3
3,2,2,2,1,1,0,0,0,0,0,2,1,0,1,0
4,0,4,0,0,0,0,0,0,1,0,0,1,0,0,0
5,5,4,3,5,1,8,3,2,1,7,2,1,1,2,1
6,2,1,2,0,2,0,1,1,3,0,0,1,0,2,1
7,3,4,4,0,1,1,1,1,0,1,1,1,1,0,2
8,4,6,2,2,0,0,1,1,2,0,0,1,0,0,0
9,1,0,0,2,2,0,1,2,0,0,0,0,2,1,1
